## Setup: Finnhub API Key

This notebook uses real market data from [Finnhub](https://finnhub.io/) (free tier: 60 API calls/min).

**To use real data:**
1. Get a free API key at https://finnhub.io/register
2. Set environment variable: `export FINNHUB_API_KEY=your_key_here`
3. Or enter the key when prompted

**Without a key:** The notebook falls back to synthetic data generation.

# Market Making — Example Notebook

Simple inventory-based market making example with equations, simulation, backtest metrics and visualisation. Includes fallback backtester if python.backtest.core is not available.

In [ ]:
import math
import random
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import json

try:
    from python.backtest.core import Backtest, sharpe_ratio, max_drawdown

# Import Finnhub helper for real market data
try:
    from python.finnhub_helper import fetch_historical_simulation, get_finnhub_api_key, create_orderbook_from_quote
    FINNHUB_AVAILABLE = True
except Exception as e:
    print(f"Finnhub helper not available: {e}")
    FINNHUB_AVAILABLE = False

except Exception:
    # minimal fallback (same as Triangular fallback)
    class Backtest:
        def __init__(self, initial_cash=100000.0):
            self.cash = initial_cash
            self.positions = {}
            self.trades = []
            self.history = []
        def execute_trade(self, timestamp, symbol, qty, price, side):
            cost = qty * price
            if side.lower() == 'buy':
                self.cash -= cost
                prev = self.positions.get(symbol, (0.0, 0.0))
                new_qty = prev[0] + qty
                new_avg = (prev[0]*prev[1] + qty*price) / (new_qty if new_qty!=0 else 1)
                self.positions[symbol] = (new_qty, new_avg)
            else:
                self.cash += cost
                prev = self.positions.get(symbol, (0.0, 0.0))
                new_qty = prev[0] - qty
                if new_qty <= 0:
                    self.positions.pop(symbol, None)
                else:
                    self.positions[symbol] = (new_qty, prev[1])
            self.trades.append({"ts": timestamp, "symbol": symbol, "qty": qty, "price": price, "side": side})
            self.mark_to_market(timestamp, {})
        def mark_to_market(self, timestamp, market_prices: dict):
            equity = self.cash
            for s,(qty,avg) in self.positions.items():
                price = market_prices.get(s, avg)
                equity += qty*price
            self.history.append((timestamp, equity))
            return equity
        def results_df(self):
            df = pd.DataFrame(self.history, columns=["ts", "equity"]).set_index("ts")
            df["returns"] = df["equity"].pct_change().fillna(0.0)
            df["cum_return"] = (1 + df["returns"]).cumprod() - 1
            return df
    def sharpe_ratio(returns, freq=252):
        if len(returns) < 2:
            return 0.0
        mu = returns.mean() * freq
        sigma = returns.std() * math.sqrt(freq)
        return mu / sigma if sigma != 0 else 0.0
    def max_drawdown(equity_curve):
        roll_max = equity_curve.cummax()
        drawdown = (equity_curve - roll_max) / roll_max
        return drawdown.min()

print('Backtest available:', 'Backtest' in globals())


## Strategy description

We post symmetric bid and ask around mid price: bid = mid * (1 - s/2), ask = mid * (1 + s/2).
We control inventory by skewing quotes: reduce size on the side that increases inventory.
Fill model: probabilistic fills based on distance to mid and volume available.

In [ ]:
# Simulation parameters
N = 500
base_mid = 100.0
spread = 0.002  # 0.2%
mm_size = 1.0
rng = random.Random(123)

bt = Backtest(100000)
inventory_limit = 20.0
skew_factor = 0.5  # reduce size on the side that increases inventory
market_prices = {}
ts = 0
for i in range(N):
    # simulate mid moves
    mid = base_mid * (1 + rng.normalvariate(0, 0.0008))
    bid = mid * (1 - spread/2)
    ask = mid * (1 + spread/2)
    # determine posted sizes with simple inventory skew
    inv = bt.positions.get('SYM', (0.0, 0.0))[0] if 'SYM' in bt.positions else 0.0
    if inv > 0:
        post_bid_size = mm_size * (1 - skew_factor)
        post_ask_size = mm_size * (1 + skew_factor)
    elif inv < 0:
        post_bid_size = mm_size * (1 + skew_factor)
        post_ask_size = mm_size * (1 - skew_factor)
    else:
        post_bid_size = post_ask_size = mm_size
    # simple fill probability: closer to mid => higher fill chance
    p_fill_bid = max(0.01, 0.5 - abs(mid - bid) / mid * 100)
    p_fill_ask = max(0.01, 0.5 - abs(ask - mid) / mid * 100)
    # random fills
    if rng.random() < p_fill_bid:
        # someone hits our bid -> we buy
        bt.execute_trade(ts, 'SYM', post_bid_size, bid, 'buy')
    if rng.random() < p_fill_ask:
        # someone lifts our ask -> we sell
        bt.execute_trade(ts, 'SYM', post_ask_size, ask, 'sell')
    # mark to market
    market_prices['SYM'] = mid
    bt.mark_to_market(ts, market_prices)
    ts += 1

df = bt.results_df()
print('Trades:', len(bt.trades))
print('Final equity:', df['equity'].iloc[-1])
print('Sharpe:', sharpe_ratio(df['returns']))
print('Max Drawdown:', max_drawdown(df['equity']))


In [ ]:
# Visualise equity and inventory over time
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['equity'], mode='lines', name='Equity'))
fig.update_layout(title='Market Making Backtest Equity', xaxis_title='t', yaxis_title='Equity')
fig.show()

if len(bt.trades) > 0:
    trades_df = pd.DataFrame(bt.trades)
    # compute inventory time series from trades
    inv = 0.0
    inv_series = []
    for r in trades_df.itertuples():
        if r.side.lower() == 'buy':
            inv += r.qty
        else:
            inv -= r.qty
        inv_series.append(inv)
    # plot last trades and inventory
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(y=inv_series, mode='lines+markers', name='Inventory'))
    fig2.update_layout(title='Inventory evolution (per trade)')
    fig2.show()
    display(trades_df.tail())
else:
    print('No fills recorded.')


### Remarks
- This is a simplified MM simulator. For production use: model orderbook depth, latency, adverse selection, maker/taker fees, and order management (cancels/replace).
- You can plug Rust connectors for fast quoting and book processing; the notebook is ready to call rust modules if they are installed.